This is a notebook to play with langgraph for agent development.

It's based on one of the versions of https://langchain-ai.github.io/langgraph/; I say one of, because the example in the README is changing quickly due to it having a lot of bugs, the bugs are different per platform (I've tried just Windows and WSL), and the bugs have incredibly cryptic error messages. For example, the call to app.invoke below, when run on Windows, gives:
    KeyError: '__pydantic_serializer__'

On WSL, the code runs, but doesn't provide the right results. The output is:
'The weather in San Francisco can be found on the National Weather Service website. You can check the current weather and forecast by visiting the following link: [San Francisco Weather Forecast](https://forecast.weather.gov/MapClick.php?lat=37.7529881&lon=-122.4174306)'

Overall, this tech seems not-yet-ready for prime-time if they can't even get a basic sample in the README file to run, so I'm pause exploring langgraph for now, move to learning MSFT's Semantic Kernel for agent development, and come back to this later when it's ready for prime-time.

In [4]:
# Prereqs (only needs to be run once)
%pip install -U langgraph langchain_openai langchain_community
#langchain_anthropic

import os
if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not set")

if os.getenv("TAVILY_API_KEY") is None:
    raise Exception("TAVILY_API_KEY not set")

#if os.getenv("ANTHROPIC_API_KEY") is None:
#    raise Exception("ANTHROPIC_API_KEY not set")

Note: you may need to restart the kernel to use updated packages.


In [6]:
from typing import Literal
from langchain_core.messages import HumanMessage
#from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
#from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

# Define the tools for the agent to use
tools = [TavilySearchResults(max_results=1)]
tool_node = ToolNode(tools)

#model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0).bind_tools(tools)
model = ChatOpenAI(temperature=0).bind_tools(tools)     # This fails on Windows, but seems to work on WSL

# Define the function that determines whether to continue or not
if END != "__end__":
    raise Exception("END is not '__end__'; please fix the code with the right value.")
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)

# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="what is the weather in sf")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

AttributeError: 'ChatCompletion' object has no attribute '__pydantic_serializer__'